<a href="https://colab.research.google.com/github/rowntreerob/gfm_poc/blob/main/audio_to_gfm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade deepgram-sdk python-dotenv openai
# dependency installs

In [2]:
# provide your own audio recording ( lang=english) 2 or 3 minutes is best
# local file .mp3  is selected & uploaded

from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  AUDIO_FILE = fn
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving out1.mp3 to out1.mp3
User uploaded file "out1.mp3" with length 7300545 bytes


In [3]:
from google.colab import files
import os

# Upload .env file with deepgram credentials, openAI credential
# see .env.example for the api keys you will need to acquire for this example
uploaded = files.upload()
env_name = list(uploaded.keys())[0]

# Rename file in colab
try:
    os.rename(env_name, '.env')
    print('File uploaded and renamed successfully.')
except:
    print('Error renaming file.')

Saving .env to .env
File uploaded and renamed successfully.


In [4]:
# deepgram api call for transcription ( STT )
import os
import httpx
from dotenv import load_dotenv
from deepgram import (
    DeepgramClient,
    ClientOptionsFromEnv,
    PrerecordedOptions,
    FileSource,
)
load_dotenv('.env')  # .env uploads w api-key, project-id values

def main():
    try:

        deepgram: DeepgramClient = DeepgramClient("", ClientOptionsFromEnv())

        # process input .mp3 file to buffer
        with open(AUDIO_FILE, "rb") as file:
            buffer_data = file.read()

        payload: FileSource = {
            "buffer": buffer_data,
        }

        options: PrerecordedOptions = PrerecordedOptions(
            model="nova-2",
            smart_format=True,
            paragraphs=True,
            punctuate=True,
        )
        # call deepgram - response is a transcribed of audio file
        response = deepgram.listen.rest.v("1").transcribe_file(
            payload, options, timeout=httpx.Timeout(300.0, connect=10.0)
        )
        data = response['results']['channels'][0]['alternatives'][0]['paragraphs']['transcript']
        # print(bucket)

    except Exception as e:
        print(f"Exception: {e}")

    finally:
    # Declare the global variable
      global transcript
    transcript = data

if __name__ == "__main__":
    main()

In [ ]:
print(transcript)

In [6]:
# placeholder call on OpenAi using simple chat.completion
# this step needs to evolve depending on the domain/ topic of the audio clip
# this step(s) goal is to curate, re-write the text from audio such that:
#    content is output that satisfies the text.body requirements
#    for a following call ... ( gfm api)
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv('.env')
api_key = os.getenv("OPENAI_API_KEY")
from openai import OpenAI


# Example usage of the API to generate text using GPT-3
client = OpenAI()
completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "developer", "content": "You are a helpful assistant."},
        {
            "role": "user",
            "content": "Write a haiku about recursion in programming."
        }
    ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='Code calls on itself,  \nWithin loops it dives deeper—  \nEndless patterns form.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)
